# Test

In [1]:
import requests
page = requests.get("https://stockbit.com/#/symbol/ICBP")
page

<Response [200]>

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
# scroll test
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")

In [8]:
from selenium.webdriver.common.by import By

driver.get("https://stockbit.com/symbol/ICBP")

comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text")

comments = list()

for index, element in enumerate(comment_elements):
    print(index, element.text, "\n")
    comments.append(element.text)

In [12]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

date = list()

for index, element in enumerate(date_elements):
    print(index, element.text, "\n")
    date.append(element.text)

0 15 Dec 22, 16:48 

1 15 Dec 22, 15:00 

2 15 Dec 22, 10:25 

3 14 Dec 22, 19:45 

4 14 Dec 22, 19:41 

5 14 Dec 22, 19:23 

6 14 Dec 22, 04:54 

7 13 Dec 22, 21:09 

8 13 Dec 22, 14:44 

9 13 Dec 22, 10:26 

10 13 Dec 22, 08:42 

11 12 Dec 22, 20:41 

12 12 Dec 22, 18:37 

13 12 Dec 22, 16:27 

14 12 Dec 22, 08:56 

15 12 Dec 22, 08:40 

16 11 Dec 22, 08:38 

17 11 Dec 22, 07:36 

18 11 Dec 22, 05:24 

19 10 Dec 22, 19:10 

20  



In [13]:
for a, b in zip([1,2], [3,4]):
    print(a,b)

1 3
2 4


In [10]:
from dateutil import parser
parser.parse("1 Oct 22, 15:00") >= parser.parse("1 Oct 22")

True

In [83]:
parser.parse("4 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29"[:16])

datetime.datetime(2022, 12, 4, 19, 42)

In [26]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

while True:
    # Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
    if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
    else:
        break
    

In [72]:
# No guarantee that Selenium will stop in 1 Oct 22.
date_elements[-2].text

'30 Sep 22, 09:22'

In [5]:
from random import randint

randint(5,10)

5

In [24]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

for symbol in symbols:
    print(f"https://stockbit.com/symbol/{symbol}")

https://stockbit.com/symbol/ICBP
https://stockbit.com/symbol/CPIN
https://stockbit.com/symbol/INDF
https://stockbit.com/symbol/MYOR
https://stockbit.com/symbol/CMRY
https://stockbit.com/symbol/GOOD
https://stockbit.com/symbol/MLBI


In [6]:
import pandas as pd

df = pd.DataFrame()
df['date'] = ['30 Sep 22, 09:22', '30 Sep 22, 09:23']
df['comment'] = ['$ICBP', '$ICBP2']
df['symbol'] = 'ICBP'

df_c = pd.DataFrame()

df_c['date'] = ['30 Sep 22, 09:22']
df_c['comment'] = ['$ICBP']
df_c['symbol'] = 'ICBP'

pd.concat([df, df_c])

,date,comment,symbol
0,"30 Sep 22, 09:22",$ICBP,ICBP
1,"30 Sep 22, 09:23",$ICBP2,ICBP
0,"30 Sep 22, 09:22",$ICBP,ICBP


In [46]:
for index, (a, b) in enumerate(zip([1,2], [3,4])):
    print(index, a, b)

0 1 3
1 2 4


# Scraping

In [29]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    # Handdle: IndexError: list index out of range
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

symbol ICBP date_elements 461 dates 460 comment_elements 461 comments 460 
symbol CPIN date_elements 201 dates 200 comment_elements 201 comments 200 
symbol INDF date_elements 401 dates 400 comment_elements 401 comments 400 
symbol MYOR date_elements 360 dates 359 comment_elements 380 comments 359 
symbol CMRY date_elements 120 dates 119 comment_elements 120 comments 119 
symbol GOOD date_elements 81 dates 80 comment_elements 81 comments 80 
symbol MLBI date_elements 101 dates 100 comment_elements 101 comments 100 


In [42]:
# <div class="stream-text-single">
#   <a class="ext-link-indicator postid-9756129" data-entity-id="9756129" href="http://bit.ly/3bLj4Oc">http://bit.ly/3bLj4Oc</a>
# </div class="stream-text-single">
# the <a class..> won't be included.
df.iloc[444]['comment']

'Lower Your Expectations\n\nKata - kata di atas saya kutip dari Charlie Munger.\n\nCharlie Munger itu kawan akrab Warren Buffett. Dia yang memperkenalkan konsep quality Investing ke Buffett. Sebelum akrab dengan Munger, Buffett adalah Grahamian sejati. Beli perusahaan di bawah tangible equities, jual ketika sudah capai fair price. Cari perusahaan yang sudah anjlok berat di bawah book value lalu jual ketika market apresiasi. Bahkan perusahaan yang tinggal 1 jumat pun bisa dibeli yang penting perusahaan itu masih punya nilai ketika dilikuidasi. Istilahnya cigarbutt atau investasi puntung rokok. Investor seperti pencari puntung rokok yang masih ada sisa dan masih nyala. Isap sekali dua kali, lalu buang.\n\nSejak kenal Munger dan baca bukunya Phil Fischer Common Stocks and Uncommon Profit, paradigma Buffett berubah.\n\nMunger dan Phil Fischer mengajarkan Buffett untuk investasi di perusahaan yang berkualitas dan punya potensi bertahan hingga 20-30 tahun ke depan. Dari pada capek - capek ja

In [44]:
df.to_csv("ICBP_MLBI.csv", index=False)